# Step 0: Imports


I need to do something simpler first.


1. Train a transformer to output 1 if token x is in the input, and 0 else.

2. Train a transformer to output 1 if token x and token y are both in the input and 0 else.

3. Train a transformer to output 1 if token x and token y are adjacent in the input.

4. Train a transformer to output 1 if token x and token y are adjacent in the input AND they're 2k, 2k+1



In [ ]:
# imports
import numpy as np
from collections import deque
import pickle
from tqdm import tqdm
np.random.seed(42)

import torch
import torch.nn as nn
import pickle
from math import sqrt
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
torch.manual_seed(42)

import os

print("imports complete")

imports complete


In [ ]:
SEQ_LEN = 32

PAD_TOKEN = 0
AVG_DEG = 2
MAX_VTXS = SEQ_LEN//AVG_DEG - 1
# vertices are labelled 1,2,...,63
# we also have a padding token which is 0.

INF = MAX_VTXS # represents unreachability

# Step 1: Generate synthetic data

In [ ]:
# original task data
NTRAIN1 = 100_000
# the data will be edge lists
# like this: [1 3 1 5 2 4 0 0 0 0]
# this represents edges (1,3), (1,5) (2,4)
# (the zeros are just padding tokens)

# the label is the shortest distance from vtx 1 to vtx 2
# or "INF" if no path exists

# fine tuning data
NTRAIN2 = 2000
# I haven't totally figured out how to do the fine tuning yet.
# So don't worry about this yet.

def random_graph(n):
    edge_list = []
    adjacencies = [set() for _ in range(n+1)]
    indices = np.random.randint(n, size=(AVG_DEG*(n-1)))+1
    for i in range(0, len(indices), 2):
        u = indices[i]
        v = indices[i + 1]
        if u != v:
            edge_list += [u,v]
            adjacencies[u].add(v)
            adjacencies[v].add(u)

    if np.random.random() < 0.25:
      edge_list += [1,2]
      adjacencies[1].add(2)
      adjacencies[2].add(1)

    edge_list += [PAD_TOKEN]*(SEQ_LEN-len(edge_list))
    return edge_list, adjacencies

"""
input: G, represented as an adjacency list
output: [INF]+[d(1,i) for i in range(n)] if target=None
if target is set to some value, then we instead just output that specific distance
"""
def SSSP(G, target=None):
    dist = [INF for _ in G]
    dist[1] = 0
    frontier = deque()
    frontier.append(1)
    while len(frontier) > 0:
        vtx = frontier.popleft()
        for x in G[vtx]:
            if dist[x] == INF:
                dist[x] = 1 + dist[vtx]
                frontier.append(x)
                if x == target:
                    return dist[target]
    if target is not None:
        return dist[target]
    else:
        return dist

def fake_SSSP(G, target=None):
    return 2 in G[1]

graphs1 = []
distance1 = []

graphs2 = []
distances2 = []

for n in tqdm(range(MAX_VTXS-1, MAX_VTXS)):
    # for _ in range(NTRAIN1//MAX_VTXS):
    for _ in range(NTRAIN1):
        edge_list, adj_list = random_graph(n)
        dist = SSSP(adj_list, target=2)

        graphs1.append(edge_list)
        distance1.append(dist)

# for n in range(8, MAX_VTXS//4):
#     for _ in range(NTRAIN2//MAX_VTXS):
#         edge_list, adj_list = random_graph(n)
#         distances = SSSP(adj_list)
#         graphs2.append(edge_list)
#         distances2.append(distances)

split1 = int(len(graphs1)*3/4)
split2 = int(len(graphs2)*3/4)

all1 = list(zip(graphs1, distance1))
np.random.shuffle(all1)
graphs1, distance1 = zip(*all1)

data = {
    "train1-data": graphs1[:split1],
    "train1-labels": distance1[:split1],
    "test1-data": graphs1[split1:],
    "test1-labels": distance1[split1:]
    # "train2-data": graphs2[:split2],
    # "train2-labels": distances2[:split2],
    # "test2-data": graphs2[split2:],
    # "test2-labels": distances2[split2:]
}

with open('data.pkl', 'wb') as file:
    pickle.dump(data, file)



100%|██████████| 1/1 [00:14<00:00, 14.42s/it]


In [ ]:
print(f"dataset size = {os.path.getsize('data.pkl')//(1024*1024)}MB")

dataset size = 45MB


# Step 2: Define Transformer Model

In [ ]:
class TransformerModel(nn.Module):
    def __init__(self, input_dim, model_dim, output_dim, num_heads, num_layers, seq_len, device, dropout=0.1):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, model_dim)
        self.model_dim = model_dim
        self.seq_len = seq_len
        self.device = device

        # weight sharing
        encoder_layer = nn.TransformerEncoderLayer(d_model=model_dim, nhead=num_heads,
                                                   dim_feedforward=model_dim*4,
                                                   dropout=dropout, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers)

        self.fc_out = nn.Linear(model_dim*seq_len, output_dim)

    # def positional_encoding(self, batch_size):
    #     pos_encoding = torch.arange(self.seq_len, device=self.device).unsqueeze(1)
    #     pos_encoding = pos_encoding.float().unsqueeze(0).repeat(batch_size, 1, 1)
    #     return pos_encoding

    def positional_encoding(self, batch_size):
        position = torch.arange(self.seq_len, dtype=torch.float, device=self.device).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, self.model_dim, 2, dtype=torch.float, device=self.device) *
                            -(torch.log(torch.tensor(500.0)) / self.model_dim))

        pos_encoding = torch.zeros(self.seq_len, self.model_dim, device=self.device)
        pos_encoding[:, 0::2] = torch.sin(position * div_term)
        pos_encoding[:, 1::2] = torch.cos(position * div_term)
        pos_encoding = pos_encoding.unsqueeze(0).repeat(batch_size, 1, 1)
        return pos_encoding

    def forward(self, src, key_padding_mask):
        batch_size, src_len = src.size(0), src.size(1)
        src_pos = self.positional_encoding(batch_size)
        embed = self.embedding(src)
        src = embed * sqrt(self.model_dim) + src_pos

        output = self.transformer_encoder(src, None, src_key_padding_mask=key_padding_mask)
        flat_output = torch.flatten(output, start_dim=1, end_dim=2)
        output = self.fc_out(flat_output)
        return output


# Step 3: Load Data

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
assert device.type == 'cuda', "CUDA is not available. Please check your GPU setup."

# PARAMS
VOCAB_SIZE = 1+MAX_VTXS # one more than the max number of vertices
MODEL_DIM = 32 # Dimension of model (embedding and transformer)
NEPOCHS = 10
BSZ = 32
LR = 0.01
NHEADS = 4
NLAYERS = 2
PAD_TOKEN = 0
model = TransformerModel(input_dim=VOCAB_SIZE, model_dim=MODEL_DIM,
                         output_dim=1, num_heads=NHEADS,
                         num_layers=NLAYERS, seq_len=SEQ_LEN,
                         device=device).to(device)

with open("data.pkl", "rb") as f:
    data = pickle.load(f)

train_data1 = data["train1-data"]
train_label1 = data["train1-labels"]
train_data_tensor = torch.tensor(train_data1, dtype=torch.long, device=device)
train_label_tensor = torch.tensor(train_label1, dtype=torch.float, device=device)
train_padding_mask = (train_data_tensor != PAD_TOKEN).bool().to(device)
train_dataset = TensorDataset(train_data_tensor, train_label_tensor, train_padding_mask)
train_loader = DataLoader(train_dataset, batch_size=BSZ, shuffle=True)

test_data1 = data["test1-data"]
test_label1 = data["test1-labels"]
test_data_tensor = torch.tensor(test_data1, dtype=torch.long, device=device)
test_label_tensor = torch.tensor(test_label1, dtype=torch.float, device=device)
test_padding_mask = (test_data_tensor != PAD_TOKEN).bool().to(device)
test_dataset = TensorDataset(test_data_tensor, test_label_tensor, test_padding_mask)
test_loader = DataLoader(test_dataset, batch_size=BSZ, shuffle=True)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

train_err = []
test_err = []

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters in the model: {trainable_params//1000}K")

train_baseline = ((train_label_tensor - train_label_tensor.mean())**2).mean().item()
print(f"train BASELINEs: {train_baseline:.4f}")

Trainable parameters in the model: 26K
train BASELINEs: 39.4069


# Dad reccomended having more "partial progress measures" / having the model "show it's work".
# or creating a different easier training regiment to start.

# Step 4: Train the Model for the first task

In [ ]:
for epoch in range(NEPOCHS):
    model.train() # set to training mode
    train_loss = 0

    for batch_src, batch_labels, batch_padding_mask in train_loader:
        optimizer.zero_grad()
        output = model(batch_src, batch_padding_mask)
        loss = criterion(output.squeeze(1), batch_labels)
        train_loss += loss.item()/len(train_loader)
        loss.backward()
        optimizer.step()

    # Evaluate performance
    model.eval()
    test_loss = 0

    with torch.no_grad():
        for batch_src, batch_labels, batch_padding_mask in test_loader:
            output = model(batch_src, batch_padding_mask)
            loss = criterion(output.squeeze(1), batch_labels)
            test_loss += loss.item()/len(test_loader)

    test_err.append(test_loss)
    train_err.append(train_loss)
    print(f"Epoch {epoch + 1}/{NEPOCHS} \t Train Err: {train_loss:.4f} \t Test Err: {test_loss:.4f} \t baseline err: {train_baseline:.4f}")

plt.figure(figsize=(10, 5))
plt.plot(test_err, label='Test', color='red')
plt.plot(train_err, label='Train', color='blue')
plt.title('Accuracy vs Epochs')
plt.xlabel('Epochs'); plt.ylabel('Accuracy')
plt.legend(); plt.grid()
plt.show()

Epoch 1/10 	 Train Err: 28.8616 	 Test Err: 39.4354 	 baseline err: 39.4069
Epoch 2/10 	 Train Err: 39.8088 	 Test Err: 39.4255 	 baseline err: 39.4069
Epoch 3/10 	 Train Err: 39.7257 	 Test Err: 39.9765 	 baseline err: 39.4069
Epoch 4/10 	 Train Err: 39.4951 	 Test Err: 40.0988 	 baseline err: 39.4069
Epoch 5/10 	 Train Err: 39.4205 	 Test Err: 39.5148 	 baseline err: 39.4069


KeyboardInterrupt: 

In [ ]:
## Q: why is this not working so well?

## maybe first try a simpler problem: just give it points for distinguishing between distance 1 or not

In [ ]:
batch_src, batch_labels, batch_padding_mask = next(iter(train_loader))
output = model(batch_src, batch_padding_mask)

In [ ]:
for x,y in zip(batch_labels.tolist(),  output.squeeze(1).tolist()):
  print(f"{int(x)} \t {y:.1f}")

1 	 5.7
3 	 5.7
15 	 7.1
1 	 5.7
2 	 7.8
15 	 7.1
1 	 0.7
15 	 5.7
5 	 5.7
1 	 5.7
1 	 0.7
4 	 5.7
2 	 7.8
3 	 5.7
3 	 5.7
15 	 7.8
15 	 7.8
1 	 5.7
3 	 7.1
1 	 5.7
3 	 5.7
1 	 7.1
1 	 7.8
2 	 5.7
1 	 5.7
15 	 7.1
6 	 7.1
1 	 5.7
1 	 5.7
1 	 5.7
15 	 7.1
1 	 7.1


In [ ]:
batch_src[2]

# Step 5: Fine Tune

# Step 6: Test generalization